# Sistema de control de existencias
## Detector de frutas en las imágenes
En este apartado se detectan las piezas de fruta con el modelo entrenado y se apuntan los resultados en un csv por imagen. Generando un histórico de piezas de fruta por imagen y dia.

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# This is needed to display the images.
%matplotlib inline

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = 'mask_rcnn_inception_resnet_v2_atrous2/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'label_map.pbtxt'

#Number of classes
NUM_CLASSES = 6

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
images=os.listdir('drive/My Drive/Test')
TEST_IMAGE_PATHS = [ os.path.join('drive/My Drive/Test', '{}'.format(i)) for i in images ]
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
import csv

IMAGE_SIZE = (12, 8)
date_c=1
with open('cantidad.csv', mode='w') as csv_file:
    fieldnames = ['date', 'image_name','apple', 'pear', 'lemon','orange','banana','pineapple']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for image_path in TEST_IMAGE_PATHS:
      file_name=os.path.basename(image_path)
      print(file_name)
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      output_dict = run_inference_for_single_image(image_np, detection_graph)
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=8,
          min_score_thresh=.4)
      
      save_in='drive/My Drive/TestRes/'+file_name
      Image.fromarray(image_np).save(save_in)

      boxes = output_dict['detection_boxes']
      max_boxes_to_draw = boxes.shape[0]
      scores = output_dict['detection_scores']
      classes = output_dict['detection_classes']
      masks = output_dict['detection_masks']
      min_score_thresh=.4
      fruit_counter=[0,0,0,0,0,0]
      fruit_counter ={'apple':0,'pear':0,'lemon':0,'orange':0,'banana':0,'pineapple':0}
      for i in range(min(max_boxes_to_draw, boxes.shape[0])):
          if scores is None or scores[i] > min_score_thresh:
              # boxes[i] is the box which will be drawn
              cat=category_index.get(classes[i])
              print ("This box is gonna get used", boxes[i], float("{:.2f}".format(scores[i]*100)), cat['name'])
              fruit_counter[cat['name']]+=1

      print("Total fruit:", fruit_counter)
      fruit_counter['date']=date_c
      fruit_counter['image_name']=file_name
      date_c+=1
      writer.writerow(fruit_counter)

## Tratamiento de los datos para la predicción de compra

In [ ]:
fruits=['apple', 'pear', 'lemon', 'banana', 'pineapple', 'orange']

import pandas as pd 
# Read data from file 'cantidad.csv' 
data = pd.read_csv("cantidad.csv") 
# Preview the first 5 lines of the loaded data 
data.head()

Pasamos los datos numéricos a integer, para poder operar con ellos. 

In [ ]:
for i in fruits:
    data[i]=data[i].astype('int32')

Generamos la tabla de consumo, calculando la diferencia entre un dia y su anterior. 

In [ ]:
data_diff=data.copy()
for i in fruits:
    data_diff[i]=data_diff[i].diff()
data_diff.head()

Generamos una tabla de compras, para crear un histórico, en la que añadiremos todos los incrementos de fruta detectados, es decir filas con valores positivos, que podrían deberse a compras realizadas.

In [ ]:
data_pos=data_diff.copy()
for i in fruits:
    data_pos.loc[data_pos[i] < 0, i] = 0

data_pos.head()

De la tabla de compras generada, eliminamos las filas en las que no se ha comprado nada, no ha habido compra. 

In [ ]:
rows_zeros=data_pos[(data_pos['apple']==0) & (data_pos['lemon']==0) & (data_pos['orange']==0) & (data_pos['pear']==0) & (data_pos['pineapple']==0) & (data_pos['banana']==0)].index

data_pos.drop(rows_zeros , inplace=True)
data_pos.head()

Procedemos a restar las piezas compradas de la tabla de consumos, de esta forma solo deberiamos ver valores iguales o inferiores a 0, en caso de detectar un valor por encima de 0 implicaria un fallo, puesto que si no se ha comprado fruta no puede haber más. Se reportará como un error y se sustituirá por un 0.

In [ ]:

for i in fruits:
    data_diff[i]=data_diff[i].sub(data_pos[i])
data_diff.head()

In [ ]:
tmp_df=data_diff[(data_diff['apple']>0) | (data_diff['lemon']>0) | (data_diff['orange']>0) | (data_diff['pear']>0) | (data_diff['pineapple']>0) | (data_diff['banana']>0)]

if not tmp_df.empty:
    print("ERROR: se ha detectado un incremento de la cantidad de fruta, \n sin haber reportado una compra. \n La información erronea es la siguiente: \n",tmp_df)
else:
    print("Datos validados")